<a href="https://colab.research.google.com/github/smishr97/Mechanism-of-Action-MoA---DeepNN/blob/main/GPU_MoA_Optimizing_NN_(using_Optuna).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install iterative-stratification

**KAGGLE - Download the dataset**

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c lish-moa
! kaggle config set -n iterativestratification
! unzip sample_submission.csv.zip
! unzip train_features.csv.zip
! unzip train_targets_nonscored.csv.zip
! unzip train_drug.csv
! unzip train_targets_scored.csv.zip
! unzip test_features.csv.zip

  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 14.9MB/s]
  0% 0.00/233k [00:00<?, ?B/s]
100% 233k/233k [00:00<00:00, 76.1MB/s]
 80% 44.0M/54.8M [00:01<00:00, 22.5MB/s]
100% 54.8M/54.8M [00:01<00:00, 39.2MB/s]
  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 85.6MB/s]
 55% 5.00M/9.16M [00:00<00:00, 15.4MB/s]
100% 9.16M/9.16M [00:00<00:00, 26.3MB/s]
  0% 0.00/535k [00:00<?, ?B/s]
100% 535k/535k [00:00<00:00, 76.2MB/s]
usage: kaggle config set [-h] -n NAME -v VALUE
kaggle config set: error: argument -v/--value is required
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_targets_nonscored.csv.zip
  inflating: train_targets_nonscored.csv  
Archive:  train_drug.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile

In [ ]:
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
if __name__ == "__main__":
    TARGET_PATH = "/content/train_targets_scored.csv"
    df = pd.read_csv(TARGET_PATH)
    df.loc[:, 'kfold'] = -1
    targets = df.drop("sig_id", axis=1).values

    mskf = MultilabelStratifiedKFold(n_splits=5)

    for fold, (train_idx, valid_idx) in enumerate(mskf.split(X=df, y=targets)):
        df.loc[valid_idx, 'kfold'] = fold

    df.to_csv("/content/train_targets_fold.csv", index=False)

In [ ]:
import torch
import pandas as pd
import numpy as np

! pip install optuna
import optuna

     |████████████████████████████████| 308 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 43.8 MB/s 
     |████████████████████████████████| 80 kB 8.9 MB/s 
     |████████████████████████████████| 75 kB 4.7 MB/s 
     |████████████████████████████████| 111 kB 51.8 MB/s 
     |████████████████████████████████| 144 kB 46.1 MB/s 
     |████████████████████████████████| 49 kB 6.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=12c48dfea863fdb3bd8236249cbe478f9691e94726ca6f8a515771b6384a17be
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import torch
import torch.nn as nn

class MoaDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x" : torch.tensor(self.features[item, :], dtype=torch.float32),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float32),
        }

# Engine class is use for evaluating and training
class Engine:
    def __init__(self, model, optimizer, device):
        self.model = model
        self.device = device
        self.optimizer = optimizer

    @staticmethod
    def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

    # Training functions
    def train(self, data_loader):
        self.model.train()
        final_loss = 0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss / len(data_loader)

    # Evaluating function
    def evaluate(self, data_loader):
        self.model.eval()
        final_loss = 0
        for data in data_loader:
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            final_loss += loss.item()
        return final_loss / len(data_loader)

class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size, num_layers, dropout):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            if len(layers) == 0:
                layers.append(nn.Linear(num_features, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)   # Layers in the list

    def forward(self, x):
        return self.model(x)

In [ ]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCS = 100


def run_training(fold, params, save_model=False):
    df = pd.read_csv('/content/train_features.csv')
    df = df.drop(["cp_type", "cp_time", "cp_dose"], axis=1)
    targets_df = pd.read_csv("/content/train_targets_fold.csv")

    feature_cols = df.drop("sig_id", axis=1).columns
    target_cols = targets_df.drop(["sig_id", "kfold"], axis=1).columns

    df = df.merge(targets_df, on="sig_id", how="left")

    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)

    xtrain = train_df[feature_cols].to_numpy()
    ytrain = train_df[target_cols].to_numpy()

    xvalid = valid_df[feature_cols].to_numpy()
    yvalid = valid_df[target_cols].to_numpy()

    train_dataset = MoaDataset(features=xtrain, targets=ytrain)
    valid_dataset = MoaDataset(features=xvalid, targets=yvalid)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1024, num_workers=8, shuffle=True
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=1024, num_workers=8
    )

    model = Model(
        num_features=xtrain.shape[1],
        num_targets=ytrain.shape[1],
        num_layers=params["num_layers"],
        hidden_size=params["hidden_size"],
        dropout=params["dropout"]
    )

    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=params["learning_rate"])
    eng = Engine(model, optimizer=optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0

    for epoch in range(EPOCS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(valid_loader)
        print(f"FOLD : {fold}, EPOCH : {epoch}, TRAIN_LOSS : {train_loss}, VALIDATION_LOSS : {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), f"Model_{fold}.bin")
        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
    return best_loss


# Optuna functions
def objective(trial):
    params = {
        "num_layers": trial.suggest_int("num_layers", 1, 7),
        "hidden_size": trial.suggest_int("hidden_size", 16, 2048),
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.7),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-3)
    }
    all_loss = []
    for f in range(5):
        temp_loss = run_training(f, params, save_model=False)
        all_loss.append(temp_loss)

    return np.mean(all_loss)


if __name__ == "__main__":
    study =optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    print("Best trial: ")
    trial_ = study.best_trial

    print(trial_.values)
    print(trial_.params)
    scores = 0
    for j in range(5):
        scr = run_training(j, trial_.params, save_model=True)
        scores += scr

    print(scores / 5)

[I 2021-10-30 12:10:09,660] A new study created in memory with name: no-name-04f8ae3c-fa47-4649-aca3-924548f2982c
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



FOLD : 0, EPOCH : 0, TRAIN_LOSS : 0.4222641469616639, VALIDATION_LOSS : 0.36610974073410035
FOLD : 0, EPOCH : 1, TRAIN_LOSS : 0.11866827387558787, VALIDATION_LOSS : 0.17372544705867768
FOLD : 0, EPOCH : 2, TRAIN_LOSS : 0.056306057266498864, VALIDATION_LOSS : 0.1313147783279419
FOLD : 0, EPOCH : 3, TRAIN_LOSS : 0.03995778411626816, VALIDATION_LOSS : 0.09345331490039825
FOLD : 0, EPOCH : 4, TRAIN_LOSS : 0.03324952251032779, VALIDATION_LOSS : 0.07497702836990357
FOLD : 0, EPOCH : 5, TRAIN_LOSS : 0.029491504360186428, VALIDATION_LOSS : 0.06327650845050811
FOLD : 0, EPOCH : 6, TRAIN_LOSS : 0.02712403414280791, VALIDATION_LOSS : 0.05826582089066505
FOLD : 0, EPOCH : 7, TRAIN_LOSS : 0.025454770087411527, VALIDATION_LOSS : 0.05366649627685547
FOLD : 0, EPOCH : 8, TRAIN_LOSS : 0.024276094707219226, VALIDATION_LOSS : 0.046585748344659804
FOLD : 0, EPOCH : 9, TRAIN_LOSS : 0.023441114413895105, VALIDATION_LOSS : 0.04494760185480118
FOLD : 0, EPOCH : 10, TRAIN_LOSS : 0.022748112580493876, VALIDATIO

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



FOLD : 1, EPOCH : 0, TRAIN_LOSS : 0.4167147584651646, VALIDATION_LOSS : 0.3748237073421478
FOLD : 1, EPOCH : 1, TRAIN_LOSS : 0.11730356043890904, VALIDATION_LOSS : 0.193576380610466
FOLD : 1, EPOCH : 2, TRAIN_LOSS : 0.056114417353743, VALIDATION_LOSS : 0.14472731649875642
FOLD : 1, EPOCH : 3, TRAIN_LOSS : 0.03977365419268608, VALIDATION_LOSS : 0.10344277471303939
FOLD : 1, EPOCH : 4, TRAIN_LOSS : 0.03313499973400643, VALIDATION_LOSS : 0.07705451101064682
FOLD : 1, EPOCH : 5, TRAIN_LOSS : 0.02943458535561436, VALIDATION_LOSS : 0.06327292472124099
FOLD : 1, EPOCH : 6, TRAIN_LOSS : 0.02711798485956694, VALIDATION_LOSS : 0.059617765992879865
FOLD : 1, EPOCH : 7, TRAIN_LOSS : 0.025436008054959148, VALIDATION_LOSS : 0.051401933282613756
FOLD : 1, EPOCH : 8, TRAIN_LOSS : 0.024290598242690687, VALIDATION_LOSS : 0.04835226237773895
FOLD : 1, EPOCH : 9, TRAIN_LOSS : 0.023490434316428083, VALIDATION_LOSS : 0.04691922143101692
FOLD : 1, EPOCH : 10, TRAIN_LOSS : 0.02280647729180361, VALIDATION_LOSS

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



...

FOLD : 4, EPOCH : 0, TRAIN_LOSS : 0.2391584866533154, VALIDATION_LOSS : 0.07922942638397217
FOLD : 4, EPOCH : 1, TRAIN_LOSS : 0.029556361762316602, VALIDATION_LOSS : 0.029645273834466933
FOLD : 4, EPOCH : 2, TRAIN_LOSS : 0.02076023817062378, VALIDATION_LOSS : 0.023478899523615836
FOLD : 4, EPOCH : 3, TRAIN_LOSS : 0.018846658303549414, VALIDATION_LOSS : 0.022019762173295022
FOLD : 4, EPOCH : 4, TRAIN_LOSS : 0.01773392232625108, VALIDATION_LOSS : 0.020640702173113822
FOLD : 4, EPOCH : 5, TRAIN_LOSS : 0.016969502854504083, VALIDATION_LOSS : 0.019884278997778893
FOLD : 4, EPOCH : 6, TRAIN_LOSS : 0.01626966874066152, VALIDATION_LOSS : 0.019251091033220293
FOLD : 4, EPOCH : 7, TRAIN_LOSS : 0.015704864685080554, VALIDATION_LOSS : 0.018800979480147362
FOLD : 4, EPOCH : 8, TRAIN_LOSS : 0.015188280787122877, VALIDATION_LOSS : 0.018319277092814445
FOLD : 4, EPOCH : 9, TRAIN_LOSS : 0.014721127324982694, VALIDATION_LOSS : 0.01799319460988045
FOLD : 4, EPOCH : 10, TRAIN_LOSS : 0.014297110714802616, 

[I 2021-10-30 15:13:36,066] Trial 5 finished with value: 0.016116734370589254 and parameters: {'num_layers': 1, 'hidden_size': 2011, 'dropout': 0.3064318420517109, 'learning_rate': 0.0003147330765295408}. Best is trial 5 with value: 0.016116734370589254.


FOLD : 4, EPOCH : 38, TRAIN_LOSS : 0.007537820204896362, VALIDATION_LOSS : 0.016291119530797003


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



FOLD : 0, EPOCH : 0, TRAIN_LOSS : 0.4660236788423438, VALIDATION_LOSS : 0.4034537494182587
FOLD : 0, EPOCH : 1, TRAIN_LOSS : 0.16545960895325007, VALIDATION_LOSS : 0.12631300687789918
FOLD : 0, EPOCH : 2, TRAIN_LOSS : 0.07797722616478016, VALIDATION_LOSS : 0.04455066397786141
FOLD : 0, EPOCH : 3, TRAIN_LOSS : 0.048141182841439, VALIDATION_LOSS : 0.02483336627483368
FOLD : 0, EPOCH : 4, TRAIN_LOSS : 0.03489170870498607, VALIDATION_LOSS : 0.022462230920791627
FOLD : 0, EPOCH : 5, TRAIN_LOSS : 0.028758212629901737, VALIDATION_LOSS : 0.022762656584382056
FOLD : 0, EPOCH : 6, TRAIN_LOSS : 0.02583850773149415, VALIDATION_LOSS : 0.022781215235590936
FOLD : 0, EPOCH : 7, TRAIN_LOSS : 0.024175150614035756, VALIDATION_LOSS : 0.022564215958118437
FOLD : 0, EPOCH : 8, TRAIN_LOSS : 0.023035784006903048, VALIDATION_LOSS : 0.021684855967760087
FOLD : 0, EPOCH : 9, TRAIN_LOSS : 0.022113388796385965, VALIDATION_LOSS : 0.021236228197813033
FOLD : 0, EPOCH : 10, TRAIN_LOSS : 0.0214121618161076, VALIDATIO